In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
import gensim.downloader as api
from gensim.models.fasttext import load_facebook_model
import random
from difflib import SequenceMatcher
from scipy import stats
import sacrebleu
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os
import spacy
import re


# nltk.download()

German word2vec model Facebook https://fasttext.cc/docs/en/crawl-vectors.html (cc.de.300.bin)

In [2]:
# Code taken from https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [3]:
# https://docs.python.org/3/library/difflib.html
    
def changes_spread(original_tokenized, changed_tokenized, opcodes):
    start_change = -1
    end_change = -1
    for opcode in opcodes:
        if opcode[0] != 'equal':
            start_change = opcode[1]
            break
    for opcode in reversed(opcodes):
        if opcode[0] != 'equal':
            end_change = opcode[2]
            break
    return max(0, end_change-start_change)/len(changed_tokenized)

    

In [4]:
def highlight_in_capital(sentence_tokenized, highlight_positions):
    """
    Params:
        sentence_tokenized: tokenzied sentence
        highlight_positions: list of 2-sized tuples: [(p1, p2), (p3,p4), ...]
            where we want to highlight sentence[p1:p2], sentence[p3:p4]
    """
    highlighted_sentence = []
    
    last = 0  # index of the last position added to the new sentence
    for (start, stop) in highlight_positions:
        highlighted_sentence.extend(
            sentence_tokenized[last:start] + \
            [w.upper() for w in sentence_tokenized[start:stop]]
        )
        last = stop
    if last < len(sentence_tokenized):
        highlighted_sentence.extend(
            sentence_tokenized[last:]
        )
    return ' '.join(highlighted_sentence)

In [5]:
def two_chunk_changed(original_tokenized, changed_tokenized, opcodes, 
                      chunk_max_length=1, spacy_model=None, w2v_model=None):
    # Return the original and changed sentences with the chunk highlighted in capital
    # Return whether this sentence has only two chunk changes within the max length. 
    # And return the distance between the two changed chunks
    
    is_two_chunk_changed = False
    chunk_distance = pd.NA
    is_same_subtree = pd.NA
    changes_similarity = pd.NA
    
    
    
    changes_types = [o[0] for o in opcodes]
    
    # If not exactly two changes, return
    if not (all(changes_type == 'replace' or changes_type == 'equal' for changes_type in changes_types) and \
        changes_types.count('replace') == 2):
        return is_two_chunk_changed, chunk_distance, is_same_subtree, changes_similarity
    
    # Find the positions of the two changed chunks
    i_replace = [i for i, change in enumerate(changes_types) if change == "replace"]
    
    # If two changed chunks not have length less than chunk_max_length, return
    if not (opcodes[i_replace[0]][2] - opcodes[i_replace[0]][1] <= chunk_max_length and \
            opcodes[i_replace[1]][2] - opcodes[i_replace[1]][1] <= chunk_max_length):
        return is_two_chunk_changed, chunk_distance, is_same_subtree, changes_similarity
    
    # At this point, this should be a valid two_chunk within length change
    is_two_chunk_changed = True
    
    # Check if there is indeed an equal chunks in between of the two changed chunk
    # Calculate the distance between two chunks = the equal chunk in between
    i_equal_in_between = (i_replace[1] + i_replace[0]) // 2
    assert opcodes[i_equal_in_between][0] == 'equal'
    chunk_distance = opcodes[i_equal_in_between][2] - opcodes[i_equal_in_between][1]


    if spacy_model is not None: 
        # In the two_chunk_changed case when chunk_max_length=1, i.e., only two words are changed 
        # comparing to the original translation
        # Check if the two changed words are in the same sub tree of the dependency tree
        if (opcodes[i_replace[0]][4] - opcodes[i_replace[0]][3] == 1 and \
            opcodes[i_replace[1]][4] - opcodes[i_replace[1]][3] == 1):
            # Find the ancestors and children of the two changed words
            doc = spacy_model(' '.join(changed_tokenized))
            token1, token2 = None, None
            family1, family2 = None, None
            for token in doc:
                if token.text == changed_tokenized[opcodes[i_replace[0]][3]]:
                    token1 = token.text
                    family1 = list(token.ancestors) + list(token.children)
                    family1 = [t.text for t in family1]
                elif token.text == changed_tokenized[opcodes[i_replace[1]][3]]:
                    token2 = token.text
                    family2 = list(token.ancestors) + list(token.children)
                    family2 = [t.text for t in family2]

            if token1 is None or token2 is None:
                is_same_subtree = pd.NA
            else:
                if token1 in family2 or token2 in family1:
                    is_same_subtree = True
                else:
                    is_same_subtree = False


    # Calculate the senmatic similarity of the two changed words (cosine similarity in [-1, 1])
    if w2v_model is not None:
        # Can only calculate when only two single tokens are changed
        if (opcodes[i_replace[0]][4] - opcodes[i_replace[0]][3] == 1 and \
            opcodes[i_replace[1]][4] - opcodes[i_replace[1]][3] == 1 and \
            opcodes[i_replace[0]][2] - opcodes[i_replace[0]][1] == 1 and \
            opcodes[i_replace[1]][2] - opcodes[i_replace[1]][1] == 1):

            original_word_1 = original_tokenized[opcodes[i_replace[0]][1]]
            changed_word_1 = changed_tokenized[opcodes[i_replace[0]][3]]

            original_word_2 = original_tokenized[opcodes[i_replace[1]][1]]
            changed_word_2 = changed_tokenized[opcodes[i_replace[1]][3]]

            if original_word_1 in w2v_model.index_to_key and original_word_2 in w2v_model.index_to_key and \
                changed_word_1 in w2v_model.index_to_key and changed_word_2 in w2v_model.index_to_key:
                changes_similarity = [{'original_word': original_word_1, 
                                       'changed_word': changed_word_1, 
                                       'semantic_similarity': w2v_model.similarity(original_word_1, changed_word_1)},
                                      {'original_word': original_word_2,
                                       'changed_word': changed_word_2,
                                       'semantic_similarity': w2v_model.similarity(original_word_2, changed_word_2)}]


    return is_two_chunk_changed, chunk_distance, is_same_subtree, changes_similarity
    
    
def highlight_changes(original_tokenized, changed_tokenized, opcodes):
    """
    Params:
        original_tokenized: tokenized original sentence
        changed_tokenized: tokenized changed sentence
        opcodes: changes to get from `original_tokenized` to `changed_tokenized`
    Returns:
        original_sentence and changed_sentence with the changes highlighted in capital
    """
    
    highlighted_original_sentence_positions = []
    highlighted_changed_sentence_positions = []
    
    for opcode in opcodes:
        tag, i1, i2, j1, j2 = opcode[0], opcode[1], opcode[2], opcode[3], opcode[4]
        
        if tag != 'equal':
            highlighted_original_sentence_positions.append((i1, i2))
            highlighted_changed_sentence_positions.append((j1, j2))
            
    original_sentence_highlighted = highlight_in_capital(
        sentence_tokenized=original_tokenized, 
        highlight_positions=highlighted_original_sentence_positions
    )
    
    changed_sentence_highlighted = highlight_in_capital(
        sentence_tokenized=changed_tokenized, 
        highlight_positions=highlighted_changed_sentence_positions
    )
    
    return original_sentence_highlighted, changed_sentence_highlighted
    
    
def calculate_change(original, changed):
    # Return the original and changed sentences with the changes highlighted in capital
    
    original_tokenized = nltk.word_tokenize(original)
    changed_tokenized = nltk.word_tokenize(changed)
    
    opcodes = SequenceMatcher(None, original_tokenized, changed_tokenized).get_opcodes()
    
    # Convert the opcodes (displayed by word index) to changes in words
    changes = []
    for opcode in opcodes:
        tag, i1, i2, j1, j2 = opcode[0], opcode[1], opcode[2], opcode[3], opcode[4]
        if tag != 'equal':
            changes.append((tag, ' '.join(original_tokenized[i1:i2]), ' '.join(changed_tokenized[j1:j2])))
    
    return original_tokenized, changed_tokenized, opcodes, changes


In [6]:
def load_alignment(path_prefix):
    alignment_file_path = f"{path_prefix}_word_alignment.txt"
    if not os.path.isfile(alignment_file_path):
        raise RuntimeError("Alignment file not exist.")
        
    else:
        with open(alignment_file_path) as f:
            lines = [line.rstrip() for line in f]
            
        translation_alignment = []
        for line in lines:
            word_pairs = line.split()
            word_pairs = [word_pair.split('<sep>') for word_pair in word_pairs]
            translation_alignment.append(dict(word_pairs))
        return translation_alignment

In [7]:
def add_reason_of_change(alignment, changes, perturbed_src_word):
    if type(changes) != list:
        return pd.NA
    elif perturbed_src_word not in alignment.keys():
        changes[0]['change_type'] = None
        changes[1]['change_type'] = None
    elif alignment[perturbed_src_word] == changes[0]['changed_word'] and alignment[perturbed_src_word] == changes[1]['changed_word']:
        # Both changes are due to perturbation --> weird --> pass
        changes[0]['change_type'] = None
        changes[1]['change_type'] = None
    elif alignment[perturbed_src_word] != changes[0]['changed_word'] and alignment[perturbed_src_word] != changes[1]['changed_word']:
        # Both changes NOT due to perturbation --> weird --> pass
        changes[0]['change_type'] = None
        changes[1]['change_type'] = None
    elif alignment[perturbed_src_word] == changes[0]['changed_word']:
        changes[0]['change_type'] = "perturbed"
        changes[1]['change_type'] = "not_perturbed"
    elif alignment[perturbed_src_word] == changes[1]['changed_word']:
        changes[0]['change_type'] = "not_perturbed"
        changes[1]['change_type'] = "perturbed"
        
    return changes
        
        

In [8]:
def pos_tag_not_perturbed_change(changes, spacy_model):
    if type(changes) != list:
        return pd.NA
    elif changes[0]['change_type'] == "not_perturbed":
        doc = spacy_model(changes[0]['changed_word'])
        return [t.pos_ for t in doc][0]
    elif changes[1]['change_type'] == "not_perturbed":
        doc = spacy_model(changes[1]['changed_word'])
        return [t.pos_ for t in doc][0]
    return pd.NA

In [9]:
def read_output_df(dataset, perturb_type, beam, replacement_strategy, analyse_feature=True, 
                   ignore_case=False, no_of_replacements=1, chunk_max_length=1, spacy_model=None, 
                   w2v_model=None, use_alignment=False, winoMT=False, ref_available=False):
    if winoMT:
        path_prefix = "output/winoMT_asmetric/wmt19_winoMT_perturbed"
        output_df = pd.read_csv('output/winoMT_asmetric/wmt19_winoMT_perturbed_format.csv', index_col=0)  
    else:
        if no_of_replacements == 1:
            path_prefix = f"output/{dataset}/{replacement_strategy}/beam{beam}_perturb{perturb_type}/seed0/translations"
        else:
            path_prefix = f"output/{dataset}/{replacement_strategy}/beam{beam}_perturb{perturb_type}/seed0/translations_5replacements"

        output_df = pd.read_csv(f"{path_prefix}.csv", index_col=0)

        # Join to get the translation of the original sentences as well
        original_trans_path_prefix = \
            f"output/{dataset}/{replacement_strategy}/beam{beam}_perturbNone/seed0/translations"
        output_df = output_df.join(pd.read_csv(
            f"{original_trans_path_prefix}.csv", index_col=0
        )['OriginalSRC-Trans'])
        
    
    # Convert columns with sentences to str type
    cols = ['SRC', 'REF', 'SRC_perturbed', 'SRC_perturbed-Trans', 'OriginalSRC-Trans']
    if not ref_available:
        cols.remove('REF')
    output_df[cols] = output_df[cols].astype(str)
    
    if ignore_case:
        output_df[cols] = output_df[cols].applymap(lambda x: x.lower())
    
    # Reorder the columns
    if winoMT:
        cols = ['SRC', 'REF', 'original_word', 'perturbed_word', 'SRC_perturbed', 'OriginalSRC-Trans', 'SRC_perturbed-Trans', 'Bias_sample']
    elif no_of_replacements == 1:
        cols = ['SRC', 'REF', 'original_word', 'perturbed_word', 'SRC_perturbed', 'OriginalSRC-Trans', 'SRC_perturbed-Trans']
    else:
        cols = ['SRC_index', 'SRC', 'REF', 'original_word', 'perturbed_word', 'SRC_perturbed', 'OriginalSRC-Trans', 'SRC_perturbed-Trans']
    if not ref_available:
        cols.remove('REF')
    output_df = output_df[cols]
    
    if analyse_feature:
        print(f"Original df shape: {output_df.shape}")
        output_df = output_df.dropna()
        print(f"After dropping none-perturbed sentences: {output_df.dropna().shape}")
        
        
        # Calculate the changes, i.e., how to get from the original trans sentence 
        # to the changed trans sentence
        output_df['tokenized_OriginalSRC-Trans'], output_df['tokenized_SRC_perturbed-Trans'], output_df['opcodes'], output_df['changes'] \
            = zip(*output_df.apply(
                lambda x: calculate_change(x['OriginalSRC-Trans'], 
                                           x['SRC_perturbed-Trans']), axis=1
            ))
        
        
        # Highlight the changes in the trans sentences
        output_df["OriginalSRC-Trans"], output_df['SRC_perturbed-Trans'] \
            = zip(*output_df.apply(
                lambda x: highlight_changes(
                    x['tokenized_OriginalSRC-Trans'], 
                    x['tokenized_SRC_perturbed-Trans'], 
                    x['opcodes']), axis=1
            ))
        
        
        
        
        if replacement_strategy == 'word2vec_similarity':
            # SRC difference is the number of occurances of the word we perturb
            output_df["SRC-edit_distance"] = output_df.apply(lambda x: x['tokenized_OriginalSRC-Trans'].count(x['original_word']), axis=1)
        else:
            output_df["SRC-edit_distance"] = 1
        output_df['Trans-edit_distance'] =  output_df.apply(
            lambda x: levenshtein(x['tokenized_OriginalSRC-Trans'], x['tokenized_SRC_perturbed-Trans']), axis=1)
        output_df["#TransChanges-#SrcChanges"] = output_df['Trans-edit_distance'] - output_df['SRC-edit_distance']
        
        output_df["#TransChanges-#SrcChanges/SentenceLength"] = (output_df['Trans-edit_distance'] - output_df['SRC-edit_distance']) / output_df['SRC'].apply(lambda x: len(nltk.word_tokenize(x)))
        
        output_df["ChangesSpread/SentenceLength"] = output_df.apply(
            lambda x: changes_spread(x['tokenized_OriginalSRC-Trans'], 
                                     x['tokenized_SRC_perturbed-Trans'], 
                                     x['opcodes']), axis=1)
        
        
        
        # See if only two chunks within given max size are changed, 
        # and do some analysis on this special case
        output_df['TwoChunksChanged'], output_df['ChunkDistance'], \
        output_df["is_same_subtree"], output_df['changes_similarity'] \
            = zip(*output_df.apply(
                lambda x: two_chunk_changed(x['tokenized_OriginalSRC-Trans'],
                                            x['tokenized_SRC_perturbed-Trans'],
                                            x['opcodes'],
                                            chunk_max_length=chunk_max_length,
                                            spacy_model=spacy_model,
                                            w2v_model=w2v_model), axis=1
            ))

        
        
        if use_alignment:
            if not winoMT:
                original_alignment = load_alignment(original_trans_path_prefix)
                output_df['original_trans_alignment'] = [alignment for alignment in original_alignment for _ in range(no_of_replacements)]
            output_df['perturbed_trans_alignment'] = load_alignment(path_prefix)
            
            # In the case where two changes occurs and the two similarities is calculated, 
            # find out which change is due to the perturbation
            output_df['changes_similarity'] = output_df.apply(
                lambda x: add_reason_of_change(
                    alignment=x['perturbed_trans_alignment'],
                    changes=x['changes_similarity'],
                    perturbed_src_word=x['perturbed_word']
                ),
                axis=1
            )
            
            if spacy_model is not None:
                # Add POS tagging of the not-perturbed change
                output_df['not_perturbed_TGT_change_type'] = output_df['changes_similarity'].apply(
                    lambda x: pos_tag_not_perturbed_change(x, spacy_model))
            
        
        # Analyse on group of changes on the same sentence
        if no_of_replacements > 1:
            additional_col_1 = output_df.groupby(by="SRC_index", axis=0)[['Trans-edit_distance', '#TransChanges-#SrcChanges']].std()
            additional_col_2 = output_df.groupby(by="SRC_index", axis=0)[['TwoChunksChanged']].sum()
            
            output_df = output_df.join(additional_col_1, rsuffix='--SD')
            output_df = output_df.join(additional_col_2, rsuffix='--total')
        
    return output_df

    


In [11]:
perturb_type = 'content'
dataset = f'masked_{perturb_type}_covost2_for_en2de'  # 'MuST-SHE-en2fr' 'IWSLT15-en2vi' 'wmt19-newstest2019-en2de'
beam = 5
replacement_strategy = 'masking_language_model'
no_of_replacements = 5
ignore_case = False  # Only Europarls needs ignore case
chunk_max_length=1
spacy_model = spacy.load("de_core_news_sm")
# Loading these models in is time consuming
de_model = load_facebook_model("data/cc.de.300.bin").wv
# vi_model = load_facebook_model("data/cc.vi.300.bin").wv
winoMT = False

# # This overwrite the above params
# winoMT = True
# perturb_type = 'pronoun'
# no_of_replacements = 1

output = read_output_df(dataset=dataset, perturb_type=perturb_type, beam=beam, 
                        replacement_strategy=replacement_strategy, ignore_case=ignore_case,
                        no_of_replacements=no_of_replacements, chunk_max_length=chunk_max_length,
                        spacy_model=spacy_model, w2v_model=de_model, use_alignment=True, 
                        winoMT=winoMT, analyse_feature=True)

# print('BLEU score: ')
# sacrebleu.corpus_bleu(output['OriginalSRC-Trans'].tolist(), [output['REF'].tolist()]).score

Original df shape: (281250, 7)
After dropping none-perturbed sentences: (281250, 7)


# Comments

- On `wmt19-newstest2019-en2de, chunk_max_length=2`
    - 902: change to 1 SRC word leads to fixed changes of an irrelevant word
    - In many cases, the form of the verb (e.g., current or past tense) are changed --> harmful in the sense that it hurt performance score?
    - Word not being translated 
    - Spoken/written style
    - Time
    
    
- On `IWSLT15-en2vi, adjective`
    - 1003: change of 1 words consistently leads to change in subject
    
    - 1003, 145, 990 noun: same
    - 236 noun: same, funny but not sure if it is wrong
    - 308 verb same 
    
--> Quantify the verb form change by stemming/lemmatization
    
Chúng, họ, gã, cô ấy, cô ta, anh ta, hắn

Changes in the word "you"


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 9999999)
# output[output['#TransChanges-#SrcChanges'] > 10].head(5)
# output[output["ChangesSpread/SentenceLength"] > 0.85].head(20)



# Two chunks changed that consistently changed over the different replacement of a word


# output[(output["TwoChunksChanged"] == True) & (output["TwoChunksChanged--total"] == 5)].sort_values(by='ChunkDistance', axis=0, ascending=False).head(1)
# output[(output["TwoChunksChanged"] == True)].sort_values(by='ChunkDistance', axis=0, ascending=False).head(100)

# Two words changed that are not in the same subtree
# output[(output["TwoChunksChanged"] == True) & (output["is_same_subtree"] == False) & (output["TwoChunksChanged--total"] == 5)]




# IWSLT15-en2vi, noun
# output.loc[[1003, 145, 990, 236]]







Sort the samples by the least similarity in changed words

In [ ]:
# Filter out the 2-word-changed cases and similarity can be calculated
def get_not_perturbed_change_similarity(changes):
    for change in changes:
        if change['change_type'] == 'not_perturbed':
            return change['semantic_similarity']
    return pd.NA

analyse_df = output[
    (output["TwoChunksChanged"] == True) & output['changes_similarity'].notna() & output['not_perturbed_TGT_change_type'].isin(['NOUN', 'VERB', 'ADJ', 'PRON'])
]
analyse_df['similarity_not_perturbed'] = analyse_df['changes_similarity'].apply(
    lambda x: get_not_perturbed_change_similarity(x)
)
analyse_df.sort_values(by='similarity_not_perturbed')[['SRC', 
                                                f'original_word', 
                                                f'perturbed_word',
                                                'OriginalSRC-Trans',
                                                f'SRC_perturbed-Trans',
                                                'ChunkDistance',
                                                'changes_similarity',
                                                'similarity_not_perturbed',
                                                'not_perturbed_TGT_change_type',
#                                                 'Bias_sample'
                                                      ]].head(50)


### Calculate metrics for detecting the bias samples

High precision --> higher chance that the returned samples are bias --> save human time

High recall --> more bias samples are retreat --> can detect more type of bias

We focus on precision then (save human cost)

In [ ]:
from sklearn.metrics import classification_report

print(' -------------------- Most-changes filter -------------------- ')
q = 20  # Take the q% sentences with the highest changes
no_changes_thresthold = np.percentile(output['#TransChanges-#SrcChanges'], 100-q)
bias_prediction = output['#TransChanges-#SrcChanges'] > no_changes_thresthold
results = classification_report(
    y_true=output['Bias_sample'], y_pred=bias_prediction, 
)
print(results)

print(' -------------------- Most-spreaded_changes filter -------------------- ')
q = 20  # Take the q% sentences with the highest spread
spread_thresthold = np.percentile(output['ChangesSpread/SentenceLength'], 100-q)
bias_prediction = output['ChangesSpread/SentenceLength'] > spread_thresthold
results = classification_report(
    y_true=output['Bias_sample'], y_pred=bias_prediction, 
)
print(results)

print(' -------------------- Two-changes filter -------------------- ')
bias_prediction = output["TwoChunksChanged"]
results = classification_report(
    y_true=output['Bias_sample'], y_pred=bias_prediction, 
)
print(results)


print(' -------------------- Two-faraway-changes filter -------------------- ')
q = 20  # Take the q% sentences with the furthest distance between 2 changes 
distance_thresthold = np.nanpercentile(output['ChunkDistance'], 100-q)
bias_prediction = output["TwoChunksChanged"] & (output['ChunkDistance'] > distance_thresthold)
results = classification_report(
    y_true=output['Bias_sample'], y_pred=bias_prediction, 
)
print(results)

print(' -------------------- Two-changes-different-subtree filter -------------------- ')
bias_prediction = output["TwoChunksChanged"] & (output["is_same_subtree"] == False)
results = classification_report(
    y_true=output['Bias_sample'], y_pred=bias_prediction, 
)
print(results)


print(' -------------------- Two-change-dissimilar filter -------------------- ')
q = 90  # Take the q% sentences with the lowest similarity of the not-perturbed change
output = output.join(analyse_df['similarity_not_perturbed'])
similiarity_threshold = np.nanpercentile(output['similarity_not_perturbed'], q)

bias_prediction = output["TwoChunksChanged"] & (output['similarity_not_perturbed'] < similiarity_threshold)
results = classification_report(
    y_true=output['Bias_sample'], y_pred=bias_prediction, 
)
print(results)

# Analyse on same original_word accross sentences

In [ ]:
output[[
    'SRC_index', 'SRC', 'original_word', 'perturbed_word', 'SRC_perturbed',
    'OriginalSRC-Trans', 'SRC_perturbed-Trans', '#TransChanges-#SrcChanges',
    '#TransChanges-#SrcChanges/SentenceLength',
    'ChangesSpread/SentenceLength', 'TwoChunksChanged', 'ChunkDistance',
    'is_same_subtree', 'changes_similarity', 'perturbed_trans_alignment',
    'not_perturbed_TGT_change_type', 'Trans-edit_distance--SD',
    '#TransChanges-#SrcChanges--SD', 'TwoChunksChanged--total'
]].groupby('original_word').mean().head()




### Most changes filter:

In [ ]:
groupped_by_word = output.groupby('original_word').mean()

q = 10  # Take the q% groups with the highest changes
no_changes_thresthold = np.percentile(groupped_by_word['#TransChanges-#SrcChanges'], 100-q)
bias_prediction = groupped_by_word['#TransChanges-#SrcChanges'] > no_changes_thresthold

bias_word_predicted = groupped_by_word[bias_prediction].index.values

output[
    output['original_word'].isin(bias_word_predicted) & \
    (output['#TransChanges-#SrcChanges'] > no_changes_thresthold)
].head(2)





### Most-spreaded_changes filter

In [ ]:
groupped_by_word = output.groupby('original_word').mean()

q = 10  # Take the q% sentences with the highest spread
spread_thresthold = np.percentile(groupped_by_word['ChangesSpread/SentenceLength'], 100-q)
bias_prediction = groupped_by_word['ChangesSpread/SentenceLength'] > spread_thresthold

bias_word_predicted = groupped_by_word[bias_prediction].index.values

output[
    output['original_word'].isin(bias_word_predicted) & \
    (output['ChangesSpread/SentenceLength'] > spread_thresthold)
].head(2)


### Two-faraway-changes filter

ACTUALLY two-changes is not a bias filter. It's just an auxilary filter to avoid paraphrasing cases. Using this we will miss out on the cases where the model has both paraphrasing and 

Here we consider in each group: the number of sentences that has 2 changes

In [ ]:
two_change_only_groupped_by_word = output[output["TwoChunksChanged"]].groupby('original_word').mean()


q = 20  # Take the q% sentences with the furthest distance between 2 changes 
distance_thresthold = np.percentile(two_change_only_groupped_by_word['ChunkDistance'], 100-q)
bias_prediction = two_change_only_groupped_by_word['ChunkDistance'] > distance_thresthold


bias_word_predicted = two_change_only_groupped_by_word[bias_prediction].index.values

output[
    output["TwoChunksChanged"] & \
    output['original_word'].isin(bias_word_predicted) & \
    (output['ChunkDistance'] > distance_thresthold)
].head(2)



### Two-changes-different-subtree filter

In [ ]:
tmp = output[output["TwoChunksChanged"] & output['is_same_subtree'].notna()]
tmp['not_same_subtree'] = 1 - tmp['is_same_subtree'].astype(int)
two_change_only_groupped_by_word = tmp.groupby('original_word').sum()

q = 20  # Take the q% groups with the highest number of different subtree changes
count_thresthold = np.percentile(two_change_only_groupped_by_word['not_same_subtree'], 100-q)
bias_prediction = two_change_only_groupped_by_word['ChunkDistance'] > count_thresthold


bias_word_predicted = two_change_only_groupped_by_word[bias_prediction].index.values

output[
    output["TwoChunksChanged"] & \
    output['original_word'].isin(bias_word_predicted) & \
    (output['is_same_subtree'] == 0)
].head(2)



### Two-change-dissimilar filter

In [ ]:
output = output.join(analyse_df['similarity_not_perturbed'])
two_change_only_groupped_by_word = output[output["TwoChunksChanged"]].groupby('original_word').mean()


q = 20  # Take the q% sentences with the lowest similarity of the not-perturbed change
similiarity_threshold = np.nanpercentile(two_change_only_groupped_by_word['similarity_not_perturbed'], q)
bias_prediction = two_change_only_groupped_by_word['similarity_not_perturbed'] < similiarity_threshold


bias_word_predicted = two_change_only_groupped_by_word[bias_prediction].index.values

output[
    output["TwoChunksChanged"] & \
    output['original_word'].isin(bias_word_predicted) & \
    (output['similarity_not_perturbed'] < similiarity_threshold)
].head(2)






In [ ]:
output.columns

## Find patterns

when a word A is replaced with B, then the change C happens

In [12]:
output.head()

,SRC_index,SRC,original_word,perturbed_word,SRC_perturbed,OriginalSRC-Trans,SRC_perturbed-Trans,tokenized_OriginalSRC-Trans,tokenized_SRC_perturbed-Trans,opcodes,...,TwoChunksChanged,ChunkDistance,is_same_subtree,changes_similarity,original_trans_alignment,perturbed_trans_alignment,not_perturbed_TGT_change_type,Trans-edit_distance--SD,#TransChanges-#SrcChanges--SD,TwoChunksChanged--total
0,0,“Don’t accept.”,accept,have,“Don’t have.”,< UNK > AKZEPTIERE nicht . < UNK >,HABEN SIE nicht .,"[<, unk, >, Akzeptiere, nicht, ., <, unk, >]","[Haben, sie, nicht, .]","[(replace, 0, 4, 0, 2), (equal, 4, 6, 2, 4), (...",...,False,<NA>,<NA>,<NA>,"{'“': '>', 'Don': 'unk', 't': 'unk', 'accept':...","{'Don': 'Haben', 'have': 'nicht', '.': '.'}",<NA>,0.547723,0.547723,0
0,0,“Don’t accept.”,accept,make,“Don’t make.”,< UNK > AKZEPTIERE NICHT . < UNK >,MACHT NICHTS .,"[<, unk, >, Akzeptiere, nicht, ., <, unk, >]","[Macht, nichts, .]","[(replace, 0, 5, 0, 2), (equal, 5, 6, 2, 3), (...",...,False,<NA>,<NA>,<NA>,"{'“': '>', 'Don': 'unk', 't': 'unk', 'accept':...","{'Don': 'Macht', 'make': 'nichts', '.': '.'}",<NA>,0.547723,0.547723,0
0,0,“Don’t accept.”,accept,know,“Don’t know.”,< UNK > AKZEPTIERE nicht . < UNK >,ICH WEISS ES nicht .,"[<, unk, >, Akzeptiere, nicht, ., <, unk, >]","[Ich, weiß, es, nicht, .]","[(replace, 0, 4, 0, 3), (equal, 4, 6, 3, 5), (...",...,False,<NA>,<NA>,<NA>,"{'“': '>', 'Don': 'unk', 't': 'unk', 'accept':...","{'know': 'weiß', '.': '.'}",<NA>,0.547723,0.547723,0
0,0,“Don’t accept.”,accept,see,“Don’t see.”,< UNK > AKZEPTIERE nicht . < UNK >,MAN KANN ES nicht SEHEN .,"[<, unk, >, Akzeptiere, nicht, ., <, unk, >]","[Man, kann, es, nicht, sehen, .]","[(replace, 0, 4, 0, 3), (equal, 4, 5, 3, 4), (...",...,False,<NA>,<NA>,<NA>,"{'“': '>', 'Don': 'unk', 't': 'unk', 'accept':...","{'t': 'nicht', 'see': 'sehen', '.': '.'}",<NA>,0.547723,0.547723,0
0,0,“Don’t accept.”,accept,do,“Don’t do.”,< UNK > AKZEPTIERE nicht . < UNK >,TUT ES ABER nicht .,"[<, unk, >, Akzeptiere, nicht, ., <, unk, >]","[Tut, es, aber, nicht, .]","[(replace, 0, 4, 0, 3), (equal, 4, 6, 3, 5), (...",...,False,<NA>,<NA>,<NA>,"{'“': '>', 'Don': 'unk', 't': 'unk', 'accept':...","{'Don': 'Tut', 'do': 'nicht', '.': '.'}",<NA>,0.547723,0.547723,0


In [13]:
output[['SRC_index', 'SRC', 'original_word', 'perturbed_word', 'SRC_perturbed',
       'OriginalSRC-Trans', 'SRC_perturbed-Trans', 'changes']].head()

,SRC_index,SRC,original_word,perturbed_word,SRC_perturbed,OriginalSRC-Trans,SRC_perturbed-Trans,changes
0,0,“Don’t accept.”,accept,have,“Don’t have.”,< UNK > AKZEPTIERE nicht . < UNK >,HABEN SIE nicht .,"[(replace, < unk > Akzeptiere, Haben sie), (de..."
0,0,“Don’t accept.”,accept,make,“Don’t make.”,< UNK > AKZEPTIERE NICHT . < UNK >,MACHT NICHTS .,"[(replace, < unk > Akzeptiere nicht, Macht nic..."
0,0,“Don’t accept.”,accept,know,“Don’t know.”,< UNK > AKZEPTIERE nicht . < UNK >,ICH WEISS ES nicht .,"[(replace, < unk > Akzeptiere, Ich weiß es), (..."
0,0,“Don’t accept.”,accept,see,“Don’t see.”,< UNK > AKZEPTIERE nicht . < UNK >,MAN KANN ES nicht SEHEN .,"[(replace, < unk > Akzeptiere, Man kann es), (..."
0,0,“Don’t accept.”,accept,do,“Don’t do.”,< UNK > AKZEPTIERE nicht . < UNK >,TUT ES ABER nicht .,"[(replace, < unk > Akzeptiere, Tut es aber), (..."


In [38]:
import string

def lower_remove_non_alphabet(input_str):
    translation = input_str.maketrans(dict.fromkeys(string.punctuation, ' '))
    return input_str.translate(translation).lower()

In [103]:
def is_due_to_perturbation(change, original_word, perturbed_word, 
                           perturbed_trans_alignment_dict, original_trans_alignment_dict):
    """
    A change in translation is directly due to perturbation if the (aligned) translation of perturbed_word
    is in changed_part AND the (aligned) translation of original_word is in original_part
    
    Params:
        change: tuple of (change_type, original_trans_part, changed_trans_part)
        original_word: original word in the SRC that was perturbed
        perturbed_word: the replacement of the original word
        perturbed_trans_alignment_dict: {src_word1:trans_word1, src_word2:trans_word2,...} of the perturbed trans
        original_trans_alignment_dict: {src_word1:trans_word1, src_word2:trans_word2,...} of the original trans
    Return: bool, pd.NA in failed alignment case
    """
    # Turn everything to lowercase, and remove any non-alphabet characters
    change_type, original_trans_part, changed_trans_part = \
        change[0], lower_remove_non_alphabet(change[1]), lower_remove_non_alphabet(change[2])
    perturbed_trans_alignment_dict = dict(
        (lower_remove_non_alphabet(k).replace(' ', ''), lower_remove_non_alphabet(v).replace(' ', '')) for k,v in perturbed_trans_alignment_dict.items()
    )
    original_trans_alignment_dict = dict(
        (lower_remove_non_alphabet(k).replace(' ', ''), lower_remove_non_alphabet(v).replace(' ', '')) for k,v in original_trans_alignment_dict.items()
    )
    original_word = lower_remove_non_alphabet(original_word)
    perturbed_word = lower_remove_non_alphabet(perturbed_word)
    

    perturbed_word_appears_in_new_trans = pd.NA
    if perturbed_word in perturbed_trans_alignment_dict.keys():
        perturbed_word_trans = perturbed_trans_alignment_dict[perturbed_word]
        if perturbed_word_trans in changed_trans_part.split():
            perturbed_word_appears_in_new_trans = True
        else:
            perturbed_word_appears_in_new_trans = False
            
    # Missed-translation, or name-specific case
    if perturbed_word in changed_trans_part.split():
        perturbed_word_appears_in_new_trans = True
            

    original_word_appears_in_old_trans = pd.NA
    if original_word in original_trans_alignment_dict.keys():
        original_word_trans = original_trans_alignment_dict[original_word]
        if original_word_trans in original_trans_part.split():
            original_word_appears_in_old_trans = True
        else:
            original_word_appears_in_old_trans = False
        
        if perturbed_word in perturbed_trans_alignment_dict.keys():
            if original_word == 'fort' and perturbed_word == 'île' and change == ('replace', 'Fort-de-France', 'Île-de-France'):
                print('-------------------------')
                print(change)
                print('-' + original_word_trans + '-')
                print('-' + perturbed_word_trans + '-')
                print(original_word_appears_in_old_trans)
                print(perturbed_word_appears_in_new_trans)
                
    # Missed-translation, or name-specific case
    if original_word in original_trans_part.split():
        original_word_appears_in_old_trans = True
            
    # If perturbed_word_appears_in_new_trans or original_word_appears_in_old_trans is true, then 
    # is_due_to_perturbation is true
    if (not pd.isnull(perturbed_word_appears_in_new_trans)) and \
        (not pd.isnull(original_word_appears_in_old_trans)):
        return (perturbed_word_appears_in_new_trans or original_word_appears_in_old_trans)
    elif (pd.isnull(perturbed_word_appears_in_new_trans)) and \
        (not pd.isnull(original_word_appears_in_old_trans)):
        if original_word_appears_in_old_trans:
            return True
        else:
            return pd.NA
    elif (not pd.isnull(perturbed_word_appears_in_new_trans)) and \
        (pd.isnull(original_word_appears_in_old_trans)):
        if perturbed_word_appears_in_new_trans:
            return True
        else:
            return pd.NA
    else:
        return pd.NA
    
    
def filter_changes(group_df):
    changes = []
    
    for index, row in group_df.iterrows():
        for change in row['changes']:
            # Filter out the changes caused by perturbation
            is_due_to_perturbation_out = is_due_to_perturbation(
                                            change, 
                                            row['original_word'], 
                                            row['perturbed_word'], 
                                            row['perturbed_trans_alignment'],
                                            row['original_trans_alignment']
                                        )
            if pd.isnull(is_due_to_perturbation_out) or is_due_to_perturbation_out:
                continue
                
            # Filter out the weird <unk>
            if change == ('delete', '< unk >', '') or change == ('insert', '', '< unk >'):
                continue
                
            # Filter out the changes that are not content-related
            all_pos_tags = [t.pos_ for t in spacy_model(change[1])] + [t.pos_ for t in spacy_model(change[2])]
            content_related_tags = 'NOUN', 'VERB', 'ADJ', 'PRON'
            overlap = not set(all_pos_tags).isdisjoint(content_related_tags)
            if not overlap:
                continue
                
            changes.append(change)
            
            
    return changes



In [104]:
from collections import Counter


def find_max_freq_change(group_df):
    """
    Params: 
        group_df: the group of results that has the same original_word and perturbed_word
    """
    assert group_df['original_word'].value_counts().shape[0] == 1  # Because this function is for a single group
    assert group_df['perturbed_word'].value_counts().shape[0] == 1  # Because this function is for a single group
    
    # Filter out the changes that are not directly due to perturbation
    all_changes = filter_changes(group_df)
    
    freq_changes = Counter(all_changes)
    
    if len(freq_changes.most_common()) == 0:
        return 0
    return freq_changes.most_common(1)[0][1]

change_freq = output.groupby(
    ['original_word', 'perturbed_word'], as_index=False
).apply(find_max_freq_change).rename(columns={None: 'max_change_freq'}).sort_values(
    by='max_change_freq', ascending=False)
    

change_freq = change_freq[change_freq['perturbed_word'].apply(lambda x: x.isalpha())]

change_freq.head(10)

,original_word,perturbed_word,max_change_freq
624,california,you,21
3913,ranked,The,16
3910,ranked,Book,15
3984,reign,school,15
1675,excuse,Believe,14
3980,reign,Academy,14
3981,reign,School,14
3982,reign,University,14
2073,grade,sons,12
2072,grade,imperial,11


In [105]:
groups = output.groupby(['original_word', 'perturbed_word'])
groups_as_list = [(original_perturb, group) for original_perturb, group in groups]
re_ordered_groupes = [groups_as_list[i] for i in change_freq.index.values]

for original_perturb, group in re_ordered_groupes:
    print("----------------------")
    print(f"original SRC word: {original_perturb[0]}")
    print(f"perturbed SRC word: {original_perturb[1]}")
    all_changes = filter_changes(group)
    freq_changes = Counter(all_changes)
    print(freq_changes.most_common(2))

----------------------
original SRC word: california
perturbed SRC word: you
[(('replace', 'USA', 'Vereinigte Staaten'), 21), (('replace', 'vom Zensus', 'von der Volkszählung'), 1)]
----------------------
original SRC word: ranked
perturbed SRC word: The
[(('replace', 'Grades', 'Klasse'), 16), (('insert', '', 'Grad'), 16)]
----------------------
original SRC word: ranked
perturbed SRC word: Book
[(('insert', '', 'Grad'), 15), (('replace', 'Jiajing-Herrschaft', 'Jiajing Herrschaft'), 8)]
----------------------
original SRC word: reign
perturbed SRC word: school
[(('replace', 'Grades', 'Klasse'), 15), (('replace', 'Rang', 'Platz'), 14)]
----------------------
original SRC word: excuse
perturbed SRC word: Believe
[(('insert', '', 'mir'), 14), (('insert', '', 'uns'), 4)]
----------------------
original SRC word: reign
perturbed SRC word: Academy
[(('replace', 'Rang', 'Platz'), 14), (('replace', 'Klasse', 'Grades'), 3)]
----------------------
original SRC word: reign
perturbed SRC word: Sch

[(('replace', 'E-Mail-Adresse', 'E-Mail-Nummer'), 6), (('replace', 'St.-Blaise-Straße', 'St-Blaise-Straße'), 1)]
----------------------
original SRC word: act
perturbed SRC word: move
[(('insert', '', 'sich'), 6), (('insert', '', 'uns'), 4)]
----------------------
original SRC word: amendment
perturbed SRC word: floor
[(('replace', 'unterstützen', 'stützen'), 6), (('replace', 'unterstützen', 'ergreifen'), 5)]
----------------------
original SRC word: excuse
perturbed SRC word: forgive
[(('insert', '', 'mir'), 6), (('insert', '', 'uns'), 2)]
----------------------
original SRC word: clos
perturbed SRC word: Rue
[(('delete', 'Straße', ''), 6), (('replace', 'Straße', 'Street'), 2)]
----------------------
original SRC word: sure
perturbed SRC word: glad
[(('delete', 'mir', ''), 6), (('insert', '', 'es'), 2)]
----------------------
original SRC word: ranked
perturbed SRC word: Chapter
[(('insert', '', 'Grad'), 6), (('replace', 'belegte', 'war'), 5)]
----------------------
original SRC word:

[(('insert', '', 'sich'), 4), (('replace', 'jeder Kreuzung', 'jedem Übergang'), 1)]
----------------------
original SRC word: wanli
perturbed SRC word: study
[(('replace', 'dritten', 'dritte'), 4), (('insert', '', 'Sie'), 3)]
----------------------
original SRC word: eighteen
perturbed SRC word: The
[(('delete', 'Straße', ''), 4), (('insert', '', 'in-acht'), 1)]
----------------------
original SRC word: wanli
perturbed SRC word: school
[(('replace', 'dritten', 'dritte'), 4), (('insert', '', 'Sie'), 3)]
----------------------
original SRC word: nowadays
perturbed SRC word: France
[(('delete', 'es', ''), 4), (('delete', 'Politiker', ''), 1)]
----------------------
original SRC word: fight
perturbed SRC word: defend
[(('insert', '', 'sich'), 4), (('insert', '', 'die Bewältigung einer'), 1)]
----------------------
original SRC word: threw
perturbed SRC word: lowered
[(('insert', '', 'fallen'), 4), (('replace', 'Tüten', 'Taschen herunter'), 2)]
----------------------
original SRC word: ate


[(('delete', 'Straße', ''), 3), (('replace', 'Straße', 'Street'), 3)]
----------------------
original SRC word: spent
perturbed SRC word: used
[(('insert', '', 'verbracht'), 3), (('delete', 'nicht geschlafen und', ''), 1)]
----------------------
original SRC word: prefecture
perturbed SRC word: Province
[(('replace', 'Bezirk', 'Kreis'), 3), (('replace', 'der Stadt', 'Stadtebene'), 1)]
----------------------
original SRC word: move
perturbed SRC word: come
[(('delete', 'sich', ''), 3), (('replace', 'Zeit', ', um'), 1)]
----------------------
original SRC word: told
perturbed SRC word: wants
[(('insert', '', 'wissen'), 3), (('replace', 'davon', 'darüber Bescheid wissen'), 1)]
----------------------
original SRC word: hurry
perturbed SRC word: get
[(('replace', 'sich', 'auf'), 3), (('replace', 'Kreuzaugen', 'schielen'), 1)]
----------------------
original SRC word: turn
perturbed SRC word: get
[(('replace', 'Jetzt', 'Es'), 3), (('replace', 'Nun', 'Es'), 3)]
----------------------
original

[(('replace', 'er', 'sie'), 3), (('replace', 'Steuerung', 'Leitung'), 1)]
----------------------
original SRC word: hour
perturbed SRC word: day
[(('replace', 'einer', 'einem'), 3), (('replace', 'weinte', 'habe'), 1)]
----------------------
original SRC word: breton
perturbed SRC word: English
[(('replace', 'Herr', 'Mister'), 3), (('replace', 'angeboten', 'vorgeschlagen'), 1)]
----------------------
original SRC word: change
perturbed SRC word: mean
[(('delete', 'sich', ''), 3), (('replace', 'in dieser Angelegenheit', 'dazu'), 1)]
----------------------
original SRC word: vote
perturbed SRC word: speak
[(('insert', '', 'sich'), 3), (('insert', '', 'mich'), 3)]
----------------------
original SRC word: effort
perturbed SRC word: step
[(('delete', 'sich', ''), 3), (('insert', '', 'Nutzung von'), 1)]
----------------------
original SRC word: sat
perturbed SRC word: looked
[(('delete', 'sich', ''), 3), (('replace', 'einer dunklen', 'eine dunkle'), 1)]
----------------------
original SRC wo

...

[(('replace', 'sie', 'er durch'), 1), (('replace', 'durchquert hatten', 'gegangen war'), 1)]
----------------------
original SRC word: funny
perturbed SRC word: and
[(('insert', '', ', das'), 1), (('replace', 'alle halten', 'jeder hält'), 1)]
----------------------
original SRC word: fun
perturbed SRC word: it
[(('replace', 'zu bieten', 'bereitzustellen'), 1), (('insert', '', 'das war'), 1)]
----------------------
original SRC word: found
perturbed SRC word: am
[(('replace', 'sich', 'selbst'), 1), (('replace', 'Schauen', 'Sehen'), 1)]
----------------------
original SRC word: françois
perturbed SRC word: Pierre
[(('replace', 'zu zerstören', 'auszureißen'), 1), (('replace', 'Nr . 694', 'Nummer sechshundertvierundneunzig'), 1)]
----------------------
original SRC word: found
perturbed SRC word: find
[(('delete', 'sich', ''), 1), (('replace', 'an der Spitze der polnischen katholischen', 'die polnisch-katholische'), 1)]
----------------------
original SRC word: found
perturbed SRC word: ha

[(('replace', 'Die Torte', 'Der Kuchen'), 1), (('replace', 'Haben', 'Behalten'), 1)]
----------------------
original SRC word: fear
perturbed SRC word: It
[(('replace', 'den Bullen', 'der Polizei'), 1), (('replace', 'wecke', 'inspiriere'), 1)]
----------------------
original SRC word: families
perturbed SRC word: people
[(('replace', 'ihre wertvollste Sache', 'ihr wertvollstes Gut'), 1), (('replace', 'an Land gingen', 'landeten'), 1)]
----------------------
original SRC word: families
perturbed SRC word: all
[(('insert', '', 'der Nachkommen'), 1), (('replace', 'sind die', 'gehören zur'), 1)]
----------------------
original SRC word: fear
perturbed SRC word: violence
[(('replace', 'habe', 'mache'), 1), (('replace', 'verlor', 'hat'), 1)]
----------------------
original SRC word: eyes
perturbed SRC word: most
[(('replace', 'kamen', 'die'), 1), (('replace', 'Gedränge', 'Andrang'), 1)]
----------------------
original SRC word: expression
perturbed SRC word: term
[(('replace', 'bestimmt', 'f

[(('replace', 'entspricht', 'nahe kommt'), 1), (('replace', 'stellt', 'steht für'), 1)]
----------------------
original SRC word: figure
perturbed SRC word: official
[(('replace', 'entspricht', 'nahe steht'), 1), (('replace', 'stellt', 'repräsentiert'), 1)]
----------------------
original SRC word: figures
perturbed SRC word: rates
[(('replace', 'steht', 'gehört'), 1), (('replace', 'verschiedene', 'unterschiedliche'), 1)]
----------------------
original SRC word: fifteen
perturbed SRC word: one
[(('replace', 'Henri Mondor Straße', 'Henri-Mondor-Straße'), 1), (('replace', 'Tandenet-Weg', 'Tandenet-Pfad'), 1)]
----------------------
original SRC word: field
perturbed SRC word: world
[(('replace', 'stoßen', 'stehen'), 1), (('replace', 'an einigen Referenzzahlen', ', dass einige Fachkräfte fehlen'), 1)]
----------------------
original SRC word: features
perturbed SRC word: contains
[(('insert', '', 'zu erstellen'), 1), (('replace', 'verschwunden', 'verflogen'), 1)]
----------------------
o

[(('replace', 'Demnach', 'Ihren Angaben zufolge'), 1), (('replace', 'Weltraum', 'operativen Raum'), 1)]
----------------------
original SRC word: goal
perturbed SRC word: job
[(('replace', 'verhindern', 'entziehen'), 1), (('replace', 'erreichen', 'übernehmen'), 1)]
----------------------
original SRC word: girls
perturbed SRC word: children
[(('replace', 'Wir sehen uns', 'Bis'), 1), (('replace', 'jemand', 'ein vierter Taucher'), 1)]
----------------------
original SRC word: guard
perturbed SRC word: governor
[(('replace', 'Waffen', 'Gewehre'), 1), (('replace', 'bat', 'aufforderte'), 1)]
----------------------
original SRC word: grow
perturbed SRC word: get
[(('replace', 'begannen', 'fingen'), 1), (('replace', 'besteht die', 'gibt es eine'), 1)]
----------------------
original SRC word: group
perturbed SRC word: ones
[(('replace', 'Einige Leute', 'Manche Menschen'), 1), (('replace', 'umgab', 'umringte'), 1)]
----------------------
original SRC word: glass
perturbed SRC word: wood
[(('re

[]
----------------------
original SRC word: thirty
perturbed SRC word: twenty
[]
----------------------
original SRC word: seventy
perturbed SRC word: eighty
[]
----------------------
original SRC word: cannot
perturbed SRC word: will
[]
----------------------
original SRC word: cannot
perturbed SRC word: should
[]
----------------------
original SRC word: cannot
perturbed SRC word: must
[]
----------------------
original SRC word: cannot
perturbed SRC word: could
[]
----------------------
original SRC word: cannot
perturbed SRC word: can
[]
----------------------
original SRC word: forty
perturbed SRC word: twenty
[]
----------------------
original SRC word: seventy
perturbed SRC word: thirty
[]
----------------------
original SRC word: seventy
perturbed SRC word: forty
[]
----------------------
original SRC word: doesn
perturbed SRC word: don
[]


# Comments

--> starts to make sense, yet have not seen bias (even gender bias)

--> A bigger dataset for inference could help?

Some correlation is good, some correlation is bad. Is it a good idea to prevent these correlation??

In [ ]:
# Statistical test to see if SRC_similarity is higher than Trans_similarity
print(output["Trans-edit_distance"].mean() - output["SRC-edit_distance"].mean())
stats.ttest_rel(output["SRC-edit_distance"], 
                output["Trans-edit_distance"], 
                alternative='less')

Tiny pvalue --> Indeed SRC-edit_distance is significantly lower than Trans-edit_distance


(Careful with this tho, bc with number of samples too large then statistical test does not make sense)

In [ ]:
_ = plt.hist(output["#TransChanges-#SrcChanges"], bins=50)

In [ ]:
print(output["ChangesSpread/SentenceLength"].describe())
output["ChangesSpread/SentenceLength"].plot.box()

Some changes seems to have the same meaning but different phrasing, e.g., noun index 24, 36, 47

Both for en-de and en-vi


Kind of bias: en-vi adjective sample 82

Should we cherry-pick examples? Or cherry-pick the replacement?


Or narrow down scope of perturbation? (e.g., on countries, jobs, gender, ...?)



Some cherry-picked examples anyway:

- He comes from England --> Ông ấy đến từ Anh
- He comes from Vietnam --> Hắn đến từ Việt Nam
- He comes from North Korea --> Hắn đến từ Bắc Triều Tiên



- He is european --> Hắn là người Châu Âu
- He is asian --> Anh ấy là người châu Á.



- He has black hair --> Hắn có tóc đen.
- He has blonde hair --> Anh ấy có tóc vàng


But if we limit this then it would hurt the model overal performance as well? 

*Jan: some kind of loss to minimize the number of changes, but not completely forbidden the changes*


# Translation quality vs #changes

In [ ]:
from nltk.translate.gleu_score import sentence_gleu

output["OriginalTran_Quality"] = output.apply(
    lambda x: sentence_gleu([nltk.word_tokenize(x['REF'])], nltk.word_tokenize(x['OriginalSRC-Trans'])), axis=1)

In [ ]:
output.plot.scatter(x='OriginalTran_Quality', y="#TransChanges-#SrcChanges/SentenceLength")

In [ ]:
np.corrcoef(output['OriginalTran_Quality'], output["#TransChanges-#SrcChanges/SentenceLength"])

In [ ]:
hist = plt.hist(output["OriginalTran_Quality"], bins='sturges')
bin_boundaries = hist[1]

In [ ]:
# # Use bins with same number of samples instead of equal-sized bins

# results, bin_boundaries = pd.qcut(output["OriginalTran_Quality"], q=5, retbins=True)
# bin_boundaries


# Remove bins with too few samples
cut_point = 99999
for i, value in enumerate(hist[0]):
    if value < 5:
        cut_point = i
        break
        
bin_boundaries = bin_boundaries[:i]



In [ ]:
bin_boundaries

X = output['OriginalTran_Quality']
Y = output["#TransChanges-#SrcChanges/SentenceLength"]

x_plot = [(bin_boundaries[i] + bin_boundaries[i+1])/2 for i in range(0, len(bin_boundaries)-1)]
y_plot = [stats.trim_mean(Y[(bin_boundaries[i] < X) & (X < bin_boundaries[i+1])], 0.1) for i in range(0, len(bin_boundaries)-1)]
plt.plot(x_plot, y_plot)
plt.xlabel('OriginalTrans_Quality')
plt.ylabel('Avg_changes')

Most of the time downward trend (not as clear for en-de with verb, adverb, pronoun; en-vi adverb, pronoun)

**Note**: the plot has outliers removed in both X and Y dimensions, by removing too small bins (X) and trimmed-mean (Y)

# #changes vs translation quality

In [ ]:
hist = plt.hist(output["#TransChanges-#SrcChanges"], bins=20)
bin_boundaries = hist[1]
hist

In [ ]:
# # Use bins with same number of samples instead of equal-sized bins
# results, bin_boundaries = pd.qcut(output["#TransChanges-#SrcChanges"], q=5, retbins=True)
# bin_boundaries


# Remove bins with too few samples
cut_point = 99999
for i, value in enumerate(hist[0]):
    if value < 10:
        cut_point = i
        break
        
bin_boundaries = bin_boundaries[:i]

In [ ]:
bin_boundaries

X = output['#TransChanges-#SrcChanges']
Y = output["OriginalTran_Quality"]

x_plot = [(bin_boundaries[i] + bin_boundaries[i+1])/2 for i in range(0, len(bin_boundaries)-1)]

y_plot = [stats.trim_mean(Y[(bin_boundaries[i] <= X) & (X <= bin_boundaries[i+1])], 0.1) for i in range(0, len(bin_boundaries)-1)]
plt.plot(x_plot, y_plot)
plt.xlabel('Avg_changes')
plt.ylabel('OriginalTran_Quality')

# SentenceLength vs #changes

In [ ]:
output['OriginalSRC-length'] = output.apply(
    lambda x: len(nltk.word_tokenize(x['SRC'])), axis=1
)

In [ ]:
output.plot.scatter(x='OriginalSRC-length', y="#TransChanges-#SrcChanges")

In [ ]:
np.corrcoef(output['OriginalSRC-length'], output["#TransChanges-#SrcChanges"])

In [ ]:
hist = plt.hist(output["OriginalSRC-length"], bins=20)
bin_boundaries = hist[1]

In [ ]:
# Remove bins with too few samples
cut_point = 99999
for i, value in enumerate(hist[0]):
    if value < 10:
        cut_point = i
        break
        
bin_boundaries = bin_boundaries[:i]

In [ ]:
X = output['OriginalSRC-length']
Y = output["#TransChanges-#SrcChanges"]

x_plot = [(bin_boundaries[i] + bin_boundaries[i+1])/2 for i in range(0, len(bin_boundaries)-1)]
y_plot = [stats.trim_mean(Y[(bin_boundaries[i] < X) & (X < bin_boundaries[i+1])], 0.1) for i in range(0, len(bin_boundaries)-1)]
plt.plot(x_plot, y_plot)
plt.xlabel('OriginalSRC-length')
plt.ylabel('Avg_changes')

# Beam_size vs #changes

In [ ]:
beam_dict = {}
beam_values = [1,2,3,4,5]
for beam in beam_values:
    beam_dict[beam] = read_output_df(dataset, perturb_type, beam, replacement_strategy)
    # Make sure the df all have the same index
    if beam > 1:
        assert beam_dict[beam].index.equals(beam_dict[beam].index)


In [ ]:
plt.plot(beam_values,
              [stats.trim_mean(beam_dict[x]['#TransChanges-#SrcChanges'], 0.1) for x in beam_values])
plt.xlabel('beam')
plt.ylabel('mean_changes')

The mean might not saying anything

In [ ]:
fig, ax = plt.subplots()
ax.boxplot([beam_dict[x]['#TransChanges-#SrcChanges'] for x in beam_values])
ax.set_xticklabels(beam_values)
ax.set_xlabel('beam')
ax.set_ylabel('#changes')

# Perturbed word type vs #changes

In [ ]:
word_type_dict = {}
word_type_values = ["noun", "verb", "adjective", "adverb", "pronoun"]
for word_type in word_type_values:
    word_type_dict[word_type] = read_output_df(dataset, perturb_type=word_type, beam=beam, replacement_strategy=replacement_strategy)

    
print('--------------------------------')
print('word type    -   trimmed-mean #changes')

for word_type in word_type_values:
    print(f"{word_type} - {stats.trim_mean(word_type_dict[word_type]['#TransChanges-#SrcChanges'], 0.1)}")


In [ ]:
fig, ax = plt.subplots()
ax.boxplot([word_type_dict[x]['#TransChanges-#SrcChanges'] for x in word_type_values])
ax.set_xticklabels(word_type_values)
ax.set_xlabel('word_type')
ax.set_ylabel('#changes')

# #Changes per sentence across word types

See if the chaos changes are sentence-specific. Excluding perturbing pronouns bc not many samples have pronoun

In [ ]:
# Find sentences that has multiple word types perturbed
word_type_values = ["noun", "verb", "adjective", "adverb"]
index_intersection = word_type_dict[word_type_values[0]].index
for i in range(1, len(word_type_values)):
    index_intersection = \
        index_intersection.intersection(word_type_dict[word_type_values[i]].index)

len(index_intersection)

In [ ]:
changes_per_word_type = pd.DataFrame()
for word_type in word_type_values:
    changes_per_word_type[word_type] = word_type_dict[word_type]["#TransChanges-#SrcChanges"].loc[index_intersection]
    
# Count the number of samples where the changes in trans always bigger than changes in SRC
changes_per_word_type[(changes_per_word_type['noun'] > 0) & (changes_per_word_type['verb'] > 0) & \
                      (changes_per_word_type['adjective'] > 0) & (changes_per_word_type['adverb'] > 0)]



Small portion of rows --> not sentence-specific

In [ ]:
import spacy 
from spacy import displacy 
nlp = spacy.load("en_core_web_sm")
sentence = "He is from Vietnam"
doc = nlp(sentence)

print(f"{'Node (from)-->':<15} {'Relation':^10} {'-->Node (to)':>15}\n")

for token in doc:
    print("{:<15} {:^10} {:>15}".format(str(token.head.text), str(token.dep_), str(token.text)))

In [ ]:
for token in doc:
    print("------------------------------------------------")
    print(f"Token: {token.text}")
    print(f"Ancestors: {list(token.ancestors)}")
    print(f"Children: {list(token.children)}")

In [ ]:
import spacy 
from spacy import displacy 
nlp = spacy.load("de_core_news_sm")
sentence = "Er kommt aus Vietnam"
doc = nlp(sentence)

print(f"{'Node (from)-->':<15} {'Relation':^10} {'-->Node (to)':>15}\n")

for token in doc:
    print("{:<15} {:^10} {:>15}".format(str(token.head.text), str(token.dep_), str(token.text)))